In [1]:
import numpy as np
import matplotlib.pyplot as plt

import time

from tqdm import tqdm

import openmm
from openmm import unit
from openmm.app.topology import Topology
from openmm.app.element import Element
import mdtraj
import csv

import config
from dham import *
from util import *

In [2]:
elem = Element(0, "X", "X", 1.0)
top = Topology()
top.addChain()
top.addResidue("xxx", top._chains[0])
top.addAtom("X1", elem, top._chains[0]._residues[0])
top.addAtom("X2", elem, top._chains[0]._residues[0])
mass = 12.0 * unit.amu
system = openmm.System() #we initialize the system every
system.addParticle(mass)
system.addParticle(mass)

1

In [3]:
def random_initial_bias3D(initial_position, num_gaussians):
    initial_position = initial_position.value_in_unit_system(openmm.unit.md_unit_system)[0]
    rng = np.random.default_rng()
    a = np.ones(num_gaussians)
    bx = rng.uniform(initial_position[0] - 0.5,initial_position[0]+0.5,num_gaussians)
    by = rng.uniform(initial_position[0] - 0.5,initial_position[0]+0.5,num_gaussians)
    bz = rng.uniform(initial_position[0] - 0.5,initial_position[0]+0.5,num_gaussians)
    cx = rng.uniform(1,5,num_gaussians)
    cy = rng.uniform(1,5,num_gaussians)
    cz = rng.uniform(1,5,num_gaussians)
    return np.concatenate((a,bx,by,bz,cx,cy,cz))

In [4]:
system, fes1 = apply_fes(system = system, 
                        particle_idx=0, 
                        gaussian_param = None, 
                        pbc = config.pbc, 
                        amp = config.amp, 
                        name = "FES",
                        mode=config.fes_mode, 
                        plot = True)

system, fes2 = apply_fes(system = system, 
                        particle_idx=1, 
                        gaussian_param = None, 
                        pbc = config.pbc, 
                        amp = config.amp, 
                        name = "FES",
                        mode=config.fes_mode, 
                        plot = True)
y1_pot = openmm.CustomExternalForce("1e2 * y^2") # reduced force constant in y for particle 1
y1_pot.addParticle(0)
z1_pot = openmm.CustomExternalForce("1e2 * z^2") # large force constant in z for particle 1
z1_pot.addParticle(0)
y2_pot, z2_pot = openmm.CustomExternalForce("1e2 * y^2"),openmm.CustomExternalForce("1e2 * y^2")
y2_pot.addParticle(1)
z2_pot.addParticle(1)


system.addForce(z1_pot) #on z, large barrier
system.addForce(y1_pot) #on y, large barrier
system.addForce(z2_pot)
system.addForce(y2_pot)

A0*exp(-(x-x00)^2/(2*sigma_x0^2))
A1*exp(-(x-x01)^2/(2*sigma_x1^2))
A2*exp(-(x-x02)^2/(2*sigma_x2^2))
A3*exp(-(x-x03)^2/(2*sigma_x3^2))
A4*exp(-(x-x04)^2/(2*sigma_x4^2))
A5*exp(-(x-x05)^2/(2*sigma_x5^2))
A6*exp(-(x-x06)^2/(2*sigma_x6^2))
A7*exp(-(x-x07)^2/(2*sigma_x7^2))
A8*exp(-(x-x08)^2/(2*sigma_x8^2))
A0*exp(-(x-x00)^2/(2*sigma_x0^2))
A1*exp(-(x-x01)^2/(2*sigma_x1^2))
A2*exp(-(x-x02)^2/(2*sigma_x2^2))
A3*exp(-(x-x03)^2/(2*sigma_x3^2))
A4*exp(-(x-x04)^2/(2*sigma_x4^2))
A5*exp(-(x-x05)^2/(2*sigma_x5^2))
A6*exp(-(x-x06)^2/(2*sigma_x6^2))
A7*exp(-(x-x07)^2/(2*sigma_x7^2))
A8*exp(-(x-x08)^2/(2*sigma_x8^2))


27

In [5]:
if config.pbc:
    a = unit.Quantity((2*np.pi*unit.nanometers, 0*unit.nanometers, 0*unit.nanometers))
    b = unit.Quantity((0*unit.nanometers, 2*np.pi*unit.nanometers, 0*unit.nanometers))
    c = unit.Quantity((0*unit.nanometers, 0*unit.nanometers, 2*np.pi*unit.nanometers)) # atom not moving in z so we set it to 1 nm
    system.setDefaultPeriodicBoxVectors(a,b,c)

#integrator
integrator = openmm.LangevinIntegrator(300*unit.kelvin, 
                                    1.0/unit.picoseconds, 
                                    0.002*unit.picoseconds)

num_propagation = int(config.sim_steps/config.propagation_step)
frame_per_propagation = int(config.propagation_step/config.dcdfreq_mfpt)
#this stores the digitized, ravelled, x, y coordinates of the particle, for every propagation.
pos_traj = np.zeros([num_propagation, frame_per_propagation]) #shape: [num_propagation, frame_per_propagation]


In [6]:
qs = []
for i in range(6):
    qs.append(np.linspace(0,2*np.pi,config.num_bins))
qspace = np.meshgrid(*qs)
x,y,dim3,dim4,dim5,dim6 = qspace

In [7]:
#x = np.linspace(0, 2*np.pi, config.num_bins)

#we start propagation.
#note num_propagation = config.sim_steps/config.propagation_step
reach = None
i_prop = 0
#for i_prop in range(num_propagation):
while reach is None:
    if i_prop >= num_propagation:
        print("propagation number exceeds num_propagation, break")
        break
    if i_prop == 0:
        print("propagation 0 starting")
        gaussian_params1 = random_initial_bias3D(initial_position = config.start_state, num_gaussians = config.num_gaussian)
        gaussian_params2 = random_intial_bias3D(initial_position = config.start_date, num_gaussians = config.num_gaussian)
        
        #global_gaussian_params = gaussian_params.reshape(1, config.num_gaussian, 3) #this is in shape [1, num_gaussian, 3]
        #we save the gaussian_params as prop_0 params. later this will be loaded in dham.
        #np.savetxt(f"./params/{time_tag}_gaussian_fes_param_{i_prop}.txt", gaussian_params)

        #we apply the initial gaussian bias (v small) to the system
        system = apply_bias(system = system, particle_idx=0, gaussian_param = gaussian_params, pbc = config.pbc, name = "BIAS", num_gaussians = config.num_gaussian)
        system = apply_bias(system = system, particle_idx=1, gaussian_param = gaussian_params, pbc = config.pbc, name = "BIAS", num_gaussians = config.num_gaussian)
        #create simulation object, this create a context object automatically.
        # when we need to pass a context object, we can pass simulation instead.
        simulation = openmm.app.Simulation(top, system, integrator, config.platform)
        simulation.context.setPositions(config.start_state)
        simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

        simulation.minimizeEnergy()
        if config.pbc:
            simulation.context.setPeriodicBoxVectors(a,b,c)

propagation 0 starting


AssertionError: gaussian_param should be in A, x0, sigma_x, format.

In [8]:
len(gaussian_params)

260

In [9]:
num_gaussians

NameError: name 'num_gaussians' is not defined

In [10]:
3 * 20

60